'/Users/henryjosephson/personal/Projects/leg_eff'

In [981]:
os.path.join(os.getcwd())

'/Users/henryjosephson/personal/Projects/leg_eff/src/leg_eff/../../'

In [991]:
import io
import logging
import os
import re
import sys

import pandas as pd
from legiscan import LegiScan

sys.path.append(os.getcwd())
from leg_eff_secrets import LEGISCAN_API_KEY

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

try:
    BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
except NameError:
    BASE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
DATA_DIR = os.path.join(BASE_DIR, "data")
RAW_DATA_DIR = os.path.join(DATA_DIR, "raw")
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, "processed")

print(BASE_DIR, DATA_DIR, RAW_DATA_DIR, PROCESSED_DATA_DIR)

# TODO: make this state-agnostic. try hardcoding if NY -> call senate api to get sponsors?


def load_dataset(state, year):
    """
    checks if datasets are in memory. loads them in if they are, downloads them if they aren't.
    only takes a single (state,year) tuple at a time - loop over it if you want more than one.
    returns a tuple of dfs: (bills, people, votes)
    """
    logger.info(f"Attempting to load dataset for {state}-{year}")

    file_path = os.path.join(RAW_DATA_DIR, f"{state}-{year}.json")

    if os.path.exists(file_path):
        logger.info("Dataset already downloaded.")
        try:
            bills_df = pd.read_json(file_path).reset_index(drop=True)
            logger.info("Dataset loaded into memory.")
            return bills_df
        except FileNotFoundError:
            logger.info("File not found.")
            pass
        except Exception as e:
            logger.error(
                "Looks like your files are downloaded, but there's something wrong. "
                + "If you aren't sure what to do, deleting the files and running"
                + " the code again is usually a safe idea."
                + f"Error: {e}"
            )

    logger.info(f"Dataset for {state}-{year} is either missing or incomplete.")

    legis = LegiScan(LEGISCAN_API_KEY)
    logger.info("Initialized LegiScan API")

    dataset_list = legis.get_dataset_list(state=state, year=year)
    logger.info(f"Retrieved dataset list for {state}-{year}")

    ACCESS_KEY = dataset_list[0]["access_key"]
    SESSION_ID = dataset_list[0]["session_id"]

    del dataset_list

    logger.info(
        "Starting dataset download. This can take my laptop up to around 5 minutes, especially for large datasets."
    )
    dataset = legis.get_dataset(access_key=ACCESS_KEY, session_id=SESSION_ID)
    del ACCESS_KEY, SESSION_ID
    assert dataset["status"] == "OK"
    logger.info("Dataset download complete")

    logger.info("Starting pre-processing.")

    readable_dataset = legis.recode_zipfile(dataset)
    namelist = readable_dataset.namelist()

    list_of_bill_dfs = []

    for file in namelist:
        if "/bill/" in file:
            content = readable_dataset.read(file)
            list_of_bill_dfs.append(
                pd.read_json(io.StringIO(content.decode("UTF-8"))).T
            )
    logger.info(f"Processed {len(list_of_bill_dfs)} bills")

    del content, file, dataset, readable_dataset, namelist

    bills_df = pd.concat(list_of_bill_dfs)

    logger.info("Pre-processing complete. Saving to disk.")

    bills_df.reset_index(drop=True).to_json(file_path, index=False)
    logger.info(f"Saved processed data to {file_path}")

    return bills_df.reset_index(drop=True)


STATE = "NY"
YEAR = 2021

ny_2021_bills = load_dataset(STATE, YEAR)
logger.info(f"Loaded {STATE} {YEAR} bills dataset")

####

# see read_senate_api.py
senate_NY_2021_bills = pd.read_json(
    os.path.join(RAW_DATA_DIR, "ny_senate_bills.json"),
)
logger.info("Loaded NY Senate 2021 bills dataset")


####


# get rid of resolutions
ny_2021_bills = ny_2021_bills[
    ny_2021_bills["bill_number"].str.startswith("A")
    | ny_2021_bills["bill_number"].str.startswith("S")
].reset_index(drop=True)
logger.info("Filtered out resolutions")

# uncomment this once you've read in the ny senate data from 2021-22:
# ny_2021_bills = ny_2021_bills.merge(
#     ny_senate_data[columns], # home rule, program bill, actual sponsor.
#     left_on="bill_number",
#     right_on="whatever the ny senate one calls its bill number",
#     how='left'
# )


# make a SAME_AS column
def get_same_as(sasts_column: pd.Series) -> pd.Series:
    return sasts_column.apply(
        # check sast `type_id` == 1 for same_as.
        lambda x: x[0]["sast_bill_number"] if len(x) > 0 else None
    )


ny_2021_bills["same_as"] = get_same_as(ny_2021_bills["sasts"])
logger.info("Created SAME_AS column")


def expand_progress(progress_list):
    progress_dict = {
        0: "N/A Pre-filed or pre-introduction",
        1: "Introduced",
        2: "Engrossed",
        3: "Enrolled",
        4: "Passed",
        5: "Vetoed",
        6: "Failed",  # Limited support based on state
        7: "Override",
        8: "Chaptered",  # what bills are chaptered?
        9: "Refer",
        10: "Report Pass",
        11: "Report DNP",
        12: "Draft",
    }

    if len(progress_list) == 0:
        return []

    templist = []

    for update in progress_list:
        templist.append(progress_dict[update["event"]].lower())
    return templist


def expand_history(history_list):
    if len(history_list) == 0:
        return []

    templist = []

    for update in history_list:
        templist.append(update["action"].lower())
    return templist


def expand_votes(vote_list):
    if len(vote_list) == 0:
        return []

    templist = []

    for update in vote_list:
        templist.append(update["desc"].lower())
    return templist


ny_2021_bills["exp_progress"] = ny_2021_bills["progress"].apply(expand_progress)
ny_2021_bills["exp_history"] = ny_2021_bills["history"].apply(expand_history)
ny_2021_bills["exp_votes"] = ny_2021_bills["votes"].apply(expand_votes)
logger.info("Expanded progress, history, and votes columns")

# get BILL
ny_2021_bills["bill"] = ny_2021_bills["exp_progress"].apply(
    lambda progList: any(
        "introduced" in x.lower() if x is not None else False for x in progList
    ),
)
logger.info("Created BILL column")

# get AIC (check history?)
ny_2021_bills["aic"] = ny_2021_bills["exp_history"].apply(
    lambda historyList: (
        any(("committee" in event) for event in historyList)
        or any(("reading" in event) for event in historyList)
        or any(("third reading" in event) for event in historyList)
        or any(("report" in event) for event in historyList)
        or any(("amend and recommit" in event) for event in historyList)
        or any(("amend (t) and recommit" in event) for event in historyList)
        # which of the following count as actions in committee?
        # or any(("enacting clause stricken" in event) for event in historyList)
        # or any(("print number" in event) for event in historyList)
        # or any(("to attorney-general for opinion" in event) for event in historyList)
        # or any(("held for consideration" in event) for event in historyList)
    )
)
logger.info("Created AIC column")

# get ABC (check history?)
# # this, as written, is just "did it make it out of committee". probably rewrite?
ny_2021_bills["out_of_committee"] = ny_2021_bills["exp_votes"].apply(
    lambda votelist: (
        any(bool(re.search("committee: favorable$", vote)) for vote in votelist)
        # so as not to count, e.g. 'assembly codes committee: favorable refer to committee rules'
    )
)
logger.info(
    "Created out_of_committee column. Henry still needs to tweak this one -- it doesn't seem to be capturing everything he wants"
)

# get PASS (check bill ID + history)
ny_2021_bills["chamber_of_origin"] = (
    ny_2021_bills["bill_number"]
    .str[0]
    .apply(lambda x: {"A": "assembly", "S": "senate"}[x])
)
logger.info("Created chamber_of_origin column")

# actually do 'passed_senate' and 'passed_assembly' columns so senators can get
# credit for bills passing senate but not for bills passing assembly
ny_2021_bills["pass"] = ny_2021_bills.apply(
    lambda row: f"passed {row['chamber_of_origin']}" in row["exp_history"],
    axis=1,
)
logger.info("Created pass column")


def standardize_bill_number_length(bill_number: str) -> str:
    bill_letter = bill_number[0]
    bill_number = bill_number[1:]

    if bool(re.search(r"[a-zA-Z]", bill_number)):
        bill_number = bill_number[:-1]

    if len(bill_number) < 5:
        bill_number = "0" * (5 - len(bill_number)) + bill_number
    return bill_letter.upper() + bill_number


ny_2021_bills["substituted_by"] = (
    # check -- does this give the same answer as using RAST?
    ny_2021_bills["exp_history"]
    .apply(
        lambda history_list: (
            history_list[-1].split()[-1]
            if any("substituted by" in x for x in history_list)
            else None
        )
    )
    .apply(lambda x: standardize_bill_number_length(x) if x is not None else None)
)
logger.info("Created substituted_by column")

# get LAW
ny_2021_bills["law"] = ny_2021_bills["exp_history"].apply(
    lambda history_list: any("signed" in x for x in history_list)
)

enacted_nos = ny_2021_bills[ny_2021_bills["law"]]["bill_number"]

ny_2021_bills["law"] = ny_2021_bills["exp_history"].apply(
    lambda history_list: any("signed" in x for x in history_list)
) | ny_2021_bills["substituted_by"].isin(enacted_nos)
logger.info("Created law column")

bill_to_spons_df = pd.DataFrame(
    {
        "bill_number": senate_NY_2021_bills["basePrintNo"].apply(
            standardize_bill_number_length
        ),
        "main_sponsor": [
            (
                d["member"]["fullName"]
                if not (d["budget"] | d["rules"] | d["redistricting"])
                else pd.Series(["budget", "rules", "redistricting"])
                .loc[[d["budget"], d["rules"], d["redistricting"]]]
                .iloc[0]
            )
            for d in senate_NY_2021_bills["sponsor"]
        ],
    }
)
ny_2021_bills = pd.merge(ny_2021_bills, bill_to_spons_df, on="bill_number", how="left")
logger.info("Merged main sponsor information")
# TODO: lots of bills give credit to 'rules' this way -- check if there's a way to
# see if there's a way to get the actual sponsor name

effectiveness_df = ny_2021_bills.groupby(by="main_sponsor")[
    ["bill", "aic", "out_of_committee", "pass", "law"]
].sum()
logger.info("Created effectiveness dataframe")

effectiveness_df["score"] = effectiveness_df.apply(
    lambda row: sum(
        (
            row["bill"] / effectiveness_df["bill"].sum(),
            row["aic"] / effectiveness_df["aic"].sum(),
            row["out_of_committee"] / effectiveness_df["out_of_committee"].sum(),
            row["pass"] / effectiveness_df["pass"].sum(),
            row["law"] / effectiveness_df["law"].sum(),
        )
    )
    / 5,
    axis=1,
)
effectiveness_df.to_csv(
    os.path.join(PROCESSED_DATA_DIR, f"{STATE} {YEAR} effectiveness-v1.csv")
)

2024-08-31 10:55:19,613 - INFO - Attempting to load dataset for NY-2021
2024-08-31 10:55:19,615 - INFO - Dataset already downloaded.


/Users/henryjosephson/personal/Projects/leg_eff /Users/henryjosephson/personal/Projects/leg_eff/data /Users/henryjosephson/personal/Projects/leg_eff/data/raw /Users/henryjosephson/personal/Projects/leg_eff/data/processed


2024-08-31 10:55:21,480 - INFO - Dataset loaded into memory.
2024-08-31 10:55:21,902 - INFO - Loaded NY 2021 bills dataset
2024-08-31 10:55:23,263 - INFO - Loaded NY Senate 2021 bills dataset
2024-08-31 10:55:23,356 - INFO - Filtered out resolutions
2024-08-31 10:55:23,362 - INFO - Created SAME_AS column
2024-08-31 10:55:27,572 - INFO - Expanded progress, history, and votes columns
2024-08-31 10:55:27,592 - INFO - Created BILL column
2024-08-31 10:55:27,660 - INFO - Created AIC column
2024-08-31 10:55:27,676 - INFO - Created out_of_committee column. Henry still needs to tweak this one -- it doesn't seem to be capturing everything he wants
2024-08-31 10:55:27,689 - INFO - Created chamber_of_origin column
2024-08-31 10:55:27,858 - INFO - Created pass column
2024-08-31 10:55:27,881 - INFO - Created substituted_by column
2024-08-31 10:55:28,061 - INFO - Created law column
2024-08-31 10:55:28,328 - INFO - Merged main sponsor information
2024-08-31 10:55:28,334 - INFO - Created effectiveness

In [974]:
ny_2021_bills["exp_history"]

0           [referred to education, referred to education]
1        [referred to consumer affairs and protection, ...
2                 [referred to cities, referred to cities]
3               [referred to housing, referred to housing]
4        [referred to governmental operations, referred...
                               ...                        
19629                                                   []
19630    [referred to rules, ordered to third reading c...
19631    [referred to finance, ordered to third reading...
19632    [referred to rules, ordered to third reading c...
19633    [referred to rules, to attorney-general for op...
Name: exp_history, Length: 19634, dtype: object

In [ ]:
ny_2021_bills.apply(
    lambda row: [x["date"] for x in row["history"]],  # after getting out of committee
    axis=1,
)

In [ ]:
[x["name"] for x in ny_2021_bills.iloc[0].sponsors if x["sponsor_type_id"] == 1]

In [ ]:
# action out of committee

ny_2021_bills["votes"].apply(
    lambda row: [
        v["date"]
        for v in row["votes"]
        if bool(re.search("committee: favorable", v["desc"].lower()))
        & (row["chamber_of_origin"] in v["desc"].lower())
    ]
)

In [ ]:
# if the date on a vote is after it gets out of committee?
# TODO: try this with action/history inst of vote
ny_2021_bills.apply(
    lambda row: row["history"] if row["out_of_committee"] else False, axis=1
)  #

In [ ]:
# ny_2021_bills['out_of_committee'] =
ny_2021_bills["votes"].apply(
    lambda row: (
        [
            row["vote"]["date"]
            for vote in row["votes"]
            if re.search("committee: favorable$", row["exp_votes"])
        ]
        # so as not to count, e.g. 'assembly codes committee: favorable refer to committee rules'
    ),
    axis=1,
)

# .apply(
#    lambda row: [x for x in [
#        vote["date"] if re.search("committee: favorable$", vote['desc']) else None
#        for vote in row["votes"]
#    ] if x],
#    axis=1,
# )  # ['exp_votes'].sample().iloc[0]

In [ ]:
ny_2021_bills["substituted_by"] = (
    # check -- does this give the same answer as using RAST?
    ny_2021_bills["exp_history"]
    .apply(
        lambda history_list: (
            history_list[-1].split()[-1]
            if any("substituted by" in x for x in history_list)
            else None
        )
    )
    .apply(lambda x: standardize_bill_number_length(x) if x is not None else None)
)

enacted_nos = ny_2021_bills[ny_2021_bills["law"]]["bill_number"]
ny_2021_bills[
    ~ny_2021_bills["bill_number"].isin(enacted_nos)
    & ny_2021_bills["substituted_by"].isin(enacted_nos)
]
# phew

In [ ]:
passed_bill_nos = [
    standardize_bill_number_length(bill["bill_number"])
    for bill in ny_2021_bills[ny_2021_bills["pass"]]
]

In [ ]:
ny_2021_bills["same_as"]

In [ ]:
for _, row in ny_2021_bills[~ny_2021_bills["pass"]].iterrows():
    if row["same_as"] in passed_bill_nos:
        print(row["same_as"])

# glad this doesn't look like it'll be a problem

In [ ]:
# ny_2023_bills['primary_sponsor'] =
ny_2023_bills["sponsors"].apply(
    lambda spons_dict: [x for x in spons_dict if x["sponsor_type_id"] == 1]
).apply(len)

# TODO: use sponsors

In [ ]:
ny_2021_bills[ny_2021_bills["exp_progress"].apply(lambda x: "chaptered" in x)][
    "bill_number"
]
# TODO: cross-check with senate api: are these all chapter amendments?
# doesn't matter too much, since this would only be chapter amendments that pass